In [10]:
pip install pmdarima

   ---------------------------------------- 0.0/613.3 kB ? eta -:--:--
   --------------------------------------- 613.3/613.3 kB 11.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 17.1 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [Cython]
   ---------------------------------------- 0/2 [Cython]
   ---------------------------------------- 0/2 [Cython]
   ---------------------------------------- 0/2 [Cython]
   -------------------- ------------------- 1/2 [pmdarima]
   -------------------- ------------------- 1/2 [pmdarima]
   ---------------------------------------- 2/2 [pmdarima]

Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from pmdarima import auto_arima
import time



In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [17]:
#data_path = '/content/drive/MyDrive/BKW/LNG Project data/' # if running this notebook in colab
data_path = '../../data/LNG Project data/' # if running this notebook locally

In [18]:
file_path = 'Prices 2009 DATA/Prices_since_2009_merged_Brent_Coal_Vol_LNG.csv'

In [19]:
# Load and prepare data
df_all = pd.read_csv(data_path + file_path)
df_all['Date'] = pd.to_datetime(df_all['Date'])
df_all.head()


,Date,Brent_Vol.,Brent_Price,CBOE_Vol.,CBOE_Price,Coal_Vol.,Coal_Price,HenryHub_Price,JKM_Vol.,JKM_Price,NBP_Price,PSV_Price,TTF_Price
0,2009-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.00
1,2009-01-02,85.07K,46.91,NaN,39.19,NaN,74.35,5.41,NaN,NaN,NaN,NaN,22.20
2,2009-01-05,109.47K,49.62,NaN,39.08,NaN,77.65,5.83,NaN,NaN,NaN,NaN,22.85
3,2009-01-06,119.83K,50.53,NaN,38.56,NaN,81.25,6.10,NaN,NaN,NaN,NaN,26.00
4,2009-01-07,130.44K,45.86,NaN,43.39,NaN,78.65,5.89,NaN,NaN,NaN,NaN,26.33


In [20]:
# get the ttf data
df_ttf = df_all[['Date', 'TTF_Price']].copy()
df_ttf.set_index('Date', inplace=True)
df_ttf = df_ttf.asfreq('D')
df_ttf['TTF_Price'] = df_ttf['TTF_Price'].interpolate() # This line fills in missing values (NaN) in the TTF_Price column by interpolating between available values.

defining a ploting function

In [26]:
import plotly.graph_objects as go
import pandas as pd

def plot_ttf_forecast_with_history(df_ttf, valid_dates, preds, actuals, horizon_days):
    # Create forecast result DataFrame
    df_plot = pd.DataFrame({
        'Forecast_Date': valid_dates,
        f'Predicted_TTF_{horizon_days}D': preds,
        f'Actual_TTF_{horizon_days}D': actuals
    }).set_index('Forecast_Date')

    # Create Plotly figure
    fig = go.Figure()

    # Add full historical price as background
    fig.add_trace(go.Scatter(
        x=df_ttf.index,
        y=df_ttf['TTF_Price'],
        mode='lines',
        name='Historical TTF Price',
        line=dict(color='lightgray', width=1),
        opacity=0.5
    ))

    # Add predicted values
    fig.add_trace(go.Scatter(
        x=df_plot.index,
        y=df_plot[f'Predicted_TTF_{horizon_days}D'],
        mode='lines+markers',
        name=f'Predicted ({horizon_days}D Ahead)',
        line=dict(color='orange')
    ))

    # Add actual values
    fig.add_trace(go.Scatter(
        x=df_plot.index,
        y=df_plot[f'Actual_TTF_{horizon_days}D'],
        mode='lines+markers',
        name='Actual Price',
        line=dict(color='blue')
    ))

    # Final layout
    fig.update_layout(
        title=f'TTF Price Forecast vs Actual ({horizon_days}-Day Horizon)',
        xaxis_title='Forecast Date',
        yaxis_title='TTF Price (EUR/MWh)',
        hovermode='x unified',
        xaxis=dict(tickangle=45)
    )

    fig.show()

Defining a model evaluation matrics

In [22]:
def forecast_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

    return {
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "R²": r2
    }


## Trying out SARIMA for TTF Price Forecasting
1. 30 days horizon, 3 years of historical data, 14 days interval

In [23]:
# 🔧 Set forecast horizon here
horizon_days = 30  # Change to 60, 90 as needed
# ---------------------------

# Forecast settings
start_forecast_date = pd.to_datetime('2020-01-01')
end_forecast_date = df_ttf.index.max() - pd.Timedelta(days=horizon_days)
forecast_dates = pd.date_range(start=start_forecast_date, end=end_forecast_date, freq='14D')

# Run expanding window forecasts
preds, actuals, valid_dates = [], [], []

for current_date in tqdm(forecast_dates):
    try:
        # 3-year expanding window
        train_start = current_date - pd.DateOffset(years=3)
        train_data = df_ttf.loc[train_start:current_date, 'TTF_Price']

        model = SARIMAX(train_data, order=(1,1,1), seasonal_order=(1,1,1,30),
                        enforce_stationarity=False, enforce_invertibility=False)
        result = model.fit(disp=False)

        forecast = result.get_forecast(steps=horizon_days)
        pred = forecast.predicted_mean.iloc[-1]
        target_date = current_date + pd.Timedelta(days=horizon_days)

        if target_date in df_ttf.index:
            actual = df_ttf.loc[target_date, 'TTF_Price']
            preds.append(pred)
            actuals.append(actual)
            valid_dates.append(current_date)

    except Exception as e:
        print(f"Error on {current_date.date()}: {e}")
        continue

# Compute MSE
mse = mean_squared_error(actuals, preds)
print(f"\nExpanding Window Forecast MSE ({horizon_days}-day horizon): {mse:.4f}")




 23%|██▎       | 33/142 [17:38<1:18:08, 43.02s/it]c:\Users\juang\anaconda3\envs\MLG_new\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
 27%|██▋       | 39/142 [22:29<1:18:13, 45.57s/it]c:\Users\juang\anaconda3\envs\MLG_new\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
 30%|██▉       | 42/142 [26:10<1:46:51, 64.12s/it]c:\Users\juang\anaconda3\envs\MLG_new\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
 30%|███       | 43/142 [28:06<2:11:39, 79.80s/it]c:\Users\juang\anaconda3\envs\MLG_new\lib\site-packages\statsmodels\base\model.py:607:


Expanding Window Forecast MSE (30-day horizon): 800.7855


In [24]:
results = forecast_metrics(actuals, preds)
for k, v in results.items():
    print(f"{k}: {v:.4f}")

MAE: 14.1388
MSE: 800.7855
RMSE: 28.2982
R²: 0.6260


In [27]:
plot_ttf_forecast_with_history(df_ttf, valid_dates, preds, actuals, horizon_days=30)

In [ ]:
# 🔧 Set forecast horizon here
horizon_days = 90  # Change if needed

# Forecast settings
start_forecast_date = pd.to_datetime('2020-01-01')
end_forecast_date = df_ttf.index.max() - pd.Timedelta(days=horizon_days)
forecast_dates = pd.date_range(start=start_forecast_date, end=end_forecast_date, freq='14D')

# Run expanding window forecasts
preds, actuals, valid_dates = [], [], []

for current_date in tqdm(forecast_dates):
    try:
        # 3-year expanding window
        train_start = current_date - pd.DateOffset(years=3)
        train_data = df_ttf.loc[train_start:current_date, 'TTF_Price']

        model = SARIMAX(train_data, order=(1,1,1), seasonal_order=(1,1,1,365),
                        enforce_stationarity=False, enforce_invertibility=False)
        result = model.fit(disp=False)

        forecast = result.get_forecast(steps=horizon_days)
        pred = forecast.predicted_mean.iloc[-1]
        target_date = current_date + pd.Timedelta(days=horizon_days)

        if target_date in df_ttf.index:
            actual = df_ttf.loc[target_date, 'TTF_Price']
            preds.append(pred)
            actuals.append(actual)
            valid_dates.append(current_date)

    except Exception as e:
        print(f"Error on {current_date.date()}: {e}")
        continue


results = forecast_metrics(actuals, preds)
for k, v in results.items():
    print(f"{k}: {v:.4f}")

 26%|██▌       | 36/138 [02:27<10:50,  6.38s/it]/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

 28%|██▊       | 39/138 [02:43<09:04,  5.50s/it]/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

 29%|██▉       | 40/138 [02:51<10:18,  6.31s/it]/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

 31%|███       | 43/138 [03:07<08:10,  5.17s/it]/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

 76%|███████▌  | 105/138 [07:28<02:09,  3.92s/it]/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelih

MAE: 24.3314
MSE: 2065.6759
RMSE: 45.4497
R²: 0.0517


In [ ]:
plot_ttf_forecast_with_history(df_ttf, valid_dates, preds, actuals, horizon_days=horizon_days)

Try 30 or 90 days horizon, **all previous years** of historical data, 14 days interval

In [ ]:
# 🔧 Set forecast horizon here
horizon_days = 30  # Change if needed

# Forecast settings
start_forecast_date = pd.to_datetime('2020-01-01')
end_forecast_date = df_ttf.index.max() - pd.Timedelta(days=horizon_days)
forecast_dates = pd.date_range(start=start_forecast_date, end=end_forecast_date, freq='14D')

# Run expanding window forecasts
preds, actuals, valid_dates = [], [], []

for current_date in tqdm(forecast_dates):
    try:
        # Use all data from the beginning
        train_start = df_ttf.index.min()  # use all data from the beginning
        train_data = df_ttf.loc[train_start:current_date, 'TTF_Price']

        model = SARIMAX(train_data, order=(1,1,1), seasonal_order=(1,1,1,365),
                        enforce_stationarity=False, enforce_invertibility=False)
        result = model.fit(disp=False)

        forecast = result.get_forecast(steps=horizon_days)
        pred = forecast.predicted_mean.iloc[-1]
        target_date = current_date + pd.Timedelta(days=horizon_days)

        if target_date in df_ttf.index:
            actual = df_ttf.loc[target_date, 'TTF_Price']
            preds.append(pred)
            actuals.append(actual)
            valid_dates.append(current_date)

    except Exception as e:
        print(f"Error on {current_date.date()}: {e}")
        continue

results = forecast_metrics(actuals, preds)
for k, v in results.items():
    print(f"{k}: {v:.4f}")


100%|██████████| 142/142 [33:38<00:00, 14.22s/it]

MAE: 13.8462
MSE: 782.7356
RMSE: 27.9774
R²: 0.6344


In [3]:
plot_ttf_forecast_with_history(df_ttf, valid_dates, preds, actuals, horizon_days=horizon_days)

NameError: name 'df_ttf' is not defined

In [9]:
# 🔧 Set forecast horizon here
horizon_days = 30  # Change if needed

# Forecast settings
start_forecast_date = pd.to_datetime('2020-01-01')
end_forecast_date = df_ttf.index.max() - pd.Timedelta(days=horizon_days)
forecast_dates = pd.date_range(start=start_forecast_date, end=end_forecast_date, freq='14D')

# Run expanding window forecasts
preds, actuals, valid_dates = [], [], []

for current_date in tqdm(forecast_dates):
    try:
        
        train_start = df_ttf.index.min()  # use all data from the beginning
        train_data = df_ttf.loc[train_start:current_date, 'TTF_Price']

        model = SARIMAX(train_data, order=(1,1,1), seasonal_order=(1,1,1,365),
                        enforce_stationarity=False, enforce_invertibility=False)
        result = model.fit(disp=False)

        forecast = result.get_forecast(steps=horizon_days)
        pred = forecast.predicted_mean.iloc[-1]
        target_date = current_date + pd.Timedelta(days=horizon_days)

        if target_date in df_ttf.index:
            actual = df_ttf.loc[target_date, 'TTF_Price']
            preds.append(pred)
            actuals.append(actual)
            valid_dates.append(current_date)

    except Exception as e:
        print(f"Error on {current_date.date()}: {e}")
        continue

results = forecast_metrics(actuals, preds)
for k, v in results.items():
    print(f"{k}: {v:.4f}")

plot_ttf_forecast_with_history(df_ttf, valid_dates, preds, actuals, horizon_days=horizon_days)

  0%|          | 0/142 [09:21<?, ?it/s]


KeyboardInterrupt: 

## Uses auto_arima from pmdarima (faster model selection)

In [14]:
# Forecast horizon
horizon_days = 30

# Forecast settings
start_forecast_date = pd.to_datetime('2020-01-01')
end_forecast_date = df_ttf.index.max() - pd.Timedelta(days=horizon_days)
forecast_dates = pd.date_range(start=start_forecast_date, end=end_forecast_date, freq='14D')

# Results containers
preds, actuals, valid_dates = [], [], []

for current_date in tqdm(forecast_dates):
    try:
        # Expanding 3-year training window
        train_start = current_date - pd.DateOffset(years=3)
        train_data = df_ttf.loc[train_start:current_date, 'TTF_Price'].dropna()

        # Skip if too few data points
        if len(train_data) < 365:  # minimum 1 year of data
            continue

        start_time = time.time()

        # Auto ARIMA with limited seasonal search
        model = auto_arima(
            train_data,
            seasonal=True,
            m=30,                    # max seasonal period (e.g., monthly)
            stepwise=True,
            suppress_warnings=True,
            error_action='ignore',
            max_order=10,
            max_p=3, max_q=3, max_P=2, max_Q=2,
            trace=False
        )

        forecast = model.predict(n_periods=horizon_days)
        pred = forecast[-1]  # take last step prediction

        target_date = current_date + pd.Timedelta(days=horizon_days)
        if target_date in df_ttf.index:
            actual = df_ttf.loc[target_date, 'TTF_Price']
            preds.append(pred)
            actuals.append(actual)
            valid_dates.append(current_date)

        # (Optional) Print timing
        print(f"✓ {current_date.date()} - Time: {time.time() - start_time:.2f}s")

    except Exception as e:
        print(f"⚠️ Error on {current_date.date()}: {e}")
        continue

# Evaluate
mse = mean_squared_error(actuals, preds)
print(f"\n✅ Expanding Window Forecast MSE ({horizon_days}-day horizon): {mse:.4f}")

  0%|          | 0/142 [00:00<?, ?it/s]C:\Users\juang\AppData\Local\Temp\ipykernel_6708\4278755324.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred = forecast[-1]  # take last step prediction
  1%|          | 1/142 [00:40<1:35:04, 40.46s/it]

✓ 2020-01-01 - Time: 40.45s


C:\Users\juang\AppData\Local\Temp\ipykernel_6708\4278755324.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred = forecast[-1]  # take last step prediction
  1%|▏         | 2/142 [01:21<1:35:29, 40.93s/it]

✓ 2020-01-15 - Time: 41.26s


C:\Users\juang\AppData\Local\Temp\ipykernel_6708\4278755324.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred = forecast[-1]  # take last step prediction
  2%|▏         | 3/142 [02:15<1:48:30, 46.84s/it]

...

✓ 2020-08-26 - Time: 9660.55s


C:\Users\juang\AppData\Local\Temp\ipykernel_6708\4278755324.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred = forecast[-1]  # take last step prediction
 13%|█▎        | 19/142 [2:54:36<70:42:47, 2069.66s/it] 

✓ 2020-09-09 - Time: 55.38s


 13%|█▎        | 19/142 [2:55:00<18:52:59, 552.68s/it] 


KeyboardInterrupt: 